In [ ]:
# Import necessary libraries
!pip install pandas_ta
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=2cbb23cd465f0d18bd37ea8ccb94fe5b61d48b3a5518634fef3812c0c9486032
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
# Function to import stock data
def import_data(stock, timeframe):
    data = yf.download(stock, period="2y", interval='1d')
    data.columns = data.columns.get_level_values(0)
    data['EMA_50'] = ta.ema(data['Close'], length=50)
    data['EMA_200'] = ta.ema(data['Close'], length=200)
    data['SMA_50'] = ta.sma(data['Close'], length=50)
    data['SMA_200'] = ta.sma(data['Close'], length=200)
    data['RSI'] = ta.rsi(data['Close'], length=14)
    data['Pct_Change'] = data['Close'].pct_change(periods=timeframe).shift(-timeframe)  # Percent change
    data.dropna(inplace=True)
    return data

# Function to prepare LSTM data
def prepare_lstm_data(data, feature_columns, target_column, time_steps=60):
    X = []
    Y = []
    for i in range(time_steps, len(data) - 5):
        X.append(data[feature_columns].iloc[i - time_steps:i].values)
        Y.append(data[target_column].iloc[i])
    return np.array(X), np.array(Y)


In [ ]:
# Function to build training data
def build_training_data(tickers, timeframe):
    feature_columns = ['Close', 'EMA_50', 'EMA_200', 'SMA_50', 'SMA_200', 'RSI']
    X_train = []
    Y_train = []
    scaler = MinMaxScaler()
    time_steps = 60

    for ticker in tickers:
        try:
            data = import_data(ticker, timeframe)
            data[feature_columns] = scaler.fit_transform(data[feature_columns])
            X, Y = prepare_lstm_data(data, feature_columns, 'Pct_Change', time_steps)
            if len(X) > 0 and len(Y) > 0:
                X_train.extend(X)
                Y_train.extend(Y)
        except Exception as e:
            print(f"Skipping {ticker} due to an error: {e}")

    return np.array(X_train), np.array(Y_train)


In [ ]:
# Function to build and train the LSTM model
def build_and_train_model(X_train, y_train):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        BatchNormalization(),
        LSTM(32),
        Dense(1)  # Regression output
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.2)
    return model


In [ ]:
# Function to predict and evaluate model performance
def predict_and_evaluate(tickers, timeframe, buy):
    # Prepare data
    X_train, y_train = build_training_data(tickers, timeframe)

    if len(X_train) == 0 or len(y_train) == 0:
        print("Training data is empty.")
        return {"tickers": [], "auc_roc": None, "accuracy": None}

    # Train model
    model = build_and_train_model(X_train, y_train)

    # Initialize variables
    feature_columns = ['Close', 'EMA_50', 'EMA_200', 'SMA_50', 'SMA_200', 'RSI']
    time_steps = 60
    scaler = MinMaxScaler()
    predictions = []
    actuals = []

    for ticker in tickers:
        try:
            data = import_data(ticker, timeframe)
            data[feature_columns] = scaler.fit_transform(data[feature_columns])
            if len(data) >= time_steps:
                last_data = data[feature_columns].iloc[-time_steps:].values
                last_data = np.expand_dims(last_data, axis=0)
                predicted_change = model.predict(last_data)[0][0]
                predictions.append(predicted_change)
                actuals.append(data['Pct_Change'].iloc[-1])
        except Exception as e:
            print(f"Error for {ticker}: {e}")

    # Evaluate metrics
    auc_roc = roc_auc_score((np.array(actuals) > 0).astype(int), predictions) if len(predictions) > 1 else None
    accuracy = accuracy_score((np.array(actuals) > 0).astype(int), (np.array(predictions) > 0).astype(int)) if len(predictions) > 1 else None

    sorted_indices = np.argsort(predictions)[::-1]  # Descending order
    sorted_tickers = [tickers[i] for i in sorted_indices]

    return {"tickers": sorted_tickers, "auc_roc": auc_roc, "accuracy": accuracy}


In [ ]:
# Main script for testing the workflow
def get_sp500_tickers():
    sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_table = pd.read_html(sp500_url)
    tickers = sp500_table[0]['Symbol'].tolist()
    return tickers
tickers = get_sp500_tickers()

timeframe = 5  # Number of days
buy = True

results = predict_and_evaluate(tickers, timeframe, buy)

# Display results
print(f"Sorted Tickers: {results['tickers']}")
print(f"AUC-ROC Score: {results['auc_roc']:.3f}" if results['auc_roc'] else "AUC-ROC: Not calculated")
print(f"Accuracy Score: {results['accuracy']:.3f}" if results['accuracy'] else "Accuracy: Not calculated")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Skipping AMTM due to an error: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required by MinMaxScaler.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Skipping BRK.B due to an error: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required by MinMaxScaler.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


[*********************100%***********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step



[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Sorted Tickers: ['MSCI', 'AOS', 'MHK', 'RJF', 'DPZ', 'JPM', 'FANG', 'BBY', 'MCHP', 'EQR', 'SJM', 'AON', 'V', 'META', 'DE', 'BX', 'SWK', 'CMS', 'MPWR', 'DLR', 'AMAT', 'LRCX', 'ACGL', 'EA', 'LEN', 'KMB', 'FDX', 'AVB', 'LMT', 'NVDA', 'AIG', 'CLX', 'PAYC', 'HST', 'CARR', 'HLT', 'GPN', 'EFX', 'DELL', 'RCL', 'MKC', 'NOW', 'AME', 'FTV', 'AMD', 'CCI', 'ORLY', 'UHS', 'PCAR', 'CBOE', 'ALGN', 'CAT', 'WTW', 'USB', 'MMC', 'TPR', 'BAC', 'PRU', 'PEG', 'HAS', 'PSA', 'ALB', 'SPGI', 'EOG', 'KIM', 'NEM', 'KVUE', 'LULU', 'FE', 'ITW', 'WYNN', 'GNRC', 'AKAM', 'AES', 'GEV', 'TRV', 'HPQ', 'LW', 'BAX', 'GS', 'WRB', 'NRG', 'ES', 'PAYX', 'SRE', 'CSX', 'CTSH', 'CTVA', 'ERIE', 'WELL', 'MDT', 'CDNS', 'DTE', 'TTWO', 'JBHT', 'EIX', 'TFC', 'FSLR', 'BXP', 'EBAY', 'AAPL', 'CL', 'PEP', 'ARE', 'URI', 'FOXA', 'D', 'ENPH', 'WDC', 'HIG', 'MRNA', 'KHC', 'BA', 'WBD', 'AEE', 'IR', 'BRO', 'INTC', 'UPS', 'UNP', 'TDY', 'EG', 'INTU', 'WAT', 'AMCR', 'BKR', 'PSX', 'PPL', 'ABBV', 'CDW', 'MET', 'GL